In [1]:
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.plotting import plot_dataset,plot_by_id
from qcodes.logger.logger import start_all_logging
# from qcodes.plots.pyqtgraph import QtPlot
from qcodes.loops import Loop
from time import sleep
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
import datetime
import time
import json
import sys
import os


In [2]:
sys.path.append('C:\QCodes_Driver_And_Scripts\Drivers')

from qcodes.instrument_drivers.yokogawa.GS200 import GS200

from Agilent_E8257D import E8257D
from RS_SMB_100A import SMB100A
from RS_SGS_100A import SGS100A
from HP3245A import HP3245A
from RS_FSV import RS_FSV
from VNA_RS import VNA


try:
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")
except KeyError as er:
    GS200.close_all() #Disconnect and irreversibly tear down the instrument
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")

try:
    FSV = RS_FSV('rs_fsv', 'TCPIP0::192.168.10.66::inst0::INSTR')
except KeyError as er:
    RS_FSV.close_all() #Disconnect and irreversibly tear down the instrument
    FSV =  RS_FSV('rs_fsv', 'TCPIP0::192.168.10.66::inst0::INSTR')
    
try:
    MW_source=E8257D('mw_source','TCPIP0::192.168.10.70::inst0::INSTR')
except KeyError as er:
    E8257D.close_all()
    MW_source=E8257D('mw_source','TCPIP0::192.168.10.70::inst0::INSTR')


Connected to: YOKOGAWA GS210 (serial:90Y438440, firmware:2.02) in 0.06s
Connected to: Rohde&Schwarz FSV3030 (serial:1330.5000K30/101857, firmware:1.80) in 0.02s
Connected to: Agilent Technologies E8257D (serial:US45470698, firmware:C.06.28) in 0.04s


In [3]:

def measure(avg,src_freq_vec, sa_freq, exp, station):

    meas = qc.Measurement(exp=exp, station=station)
    meas.register_parameter(MW_source.frequency)
    meas.register_custom_parameter(name='spectrum_mw_on',paramtype='array',setpoints=[MW_source.frequency])
    meas.register_custom_parameter(name='spectrum_mw_off',paramtype='array',setpoints=[MW_source.frequency])

    with meas.run() as datasaver:
        
        run_id=datasaver.dataset.run_id
        qc.load_by_run_spec(captured_run_id=run_id).add_metadata('parameter_snap',json.dumps(parameter_snap))
        
        for freq in tqdm(src_freq_vec):
            
            MW_source.frequency(freq*1.e9)
            
            get_f = float(MW_source.frequency.get())
            
            get_s_off  = 0
            get_s      = 0
            
            for n in tqdm(range(avg),leave=False):     
                
                MW_source.status('OFF')
                
                get_s_off =  get_s_off + np.average(1.e-3*10**(FSV.spectrum.get()/10))
                
                MW_source.status(src_status.upper())

                get_s = get_s + np.average(1.e-3*10**(FSV.spectrum.get()/10))
                
                sleep(0.01)
                
            datasaver.add_result((MW_source.frequency,get_f),("spectrum_mw_on",get_s/avg),("spectrum_mw_off",get_s_off/avg))

          


In [4]:

start_all_logging()

# Create a station
station = qc.Station()
station.add_component(FSV)
# station.add_component(pump_source)
station.add_component(I_source)
# station.add_component(I_twpa)
station.add_component(MW_source)

station.snapshot()
station.components

# Experiment details
user='DF'
date=datetime.datetime.today().strftime('%Y_%m_%d')
description='FSV_CW_vs_Agilent_freq'
database_name = date+"_"+user+"_"+description


Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.ROCH\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.ROCH\.qcodes\logs\221027-24712-qcodes.log


2022-10-27 09:37:25,531 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 269 ¦ [gs200_program(GS200Program)] Snapshot: Could not update parameter: trigger


In [5]:
script_dir=os.path.dirname(os.path.abspath("__file__"))
data_dir=os.path.join('C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\'+date)

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

data_dir=data_dir +'\\'+description

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

In [6]:

qc.initialise_or_create_database_at(data_dir+'\\'+database_name)
qc.config.core.db_location


'C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\2022_10_27\\FSV_CW_vs_Agilent_freq\\2022_10_27_DF_FSV_CW_vs_Agilent_freq'

In [7]:

current = -14.842 #mA

# # TWPA PUMP PARAMETERS
# pump_freq    = 5.209                #GHz
# pump_power   = -0.28                #dBm
# pump_status  = 'off'
# twpa_current = 2.7474747            #mA

# print('\nMeasurement with TWPA {}'.format(pump_status))


# MW SOURCE

src_center     = 2.8
src_span       = 0.5
src_points     = 200
src_frequency  = np.linspace(src_center-0.5*src_span,src_center + 0.5*src_span, src_points) #GHz
src_power      = 5
src_status     = 'on'

# Spectral Analyser parameters

FSV_rbw         = 10*1.e6 #Hz
FSV_ref         = 0       #dB
FSV_avg         = 0
FSV_att         = 0       #dB
FSV_npt         = 200
FSV_center      = 8.51     #GHz
FSV_span        = 0       #0 span mode

sweep_time_min = FSV_npt / FSV_rbw
trace_average  = 25

FSV_time_sweep  = .5
FSV_time_sweep  = max(sweep_time_min,FSV_time_sweep)


if FSV_time_sweep < sweep_time_min :
    raise ValueError("sweeptime to small with respect to the sweep points and rbw")

In [8]:

if I_source.output()=='on':
    I_source.output('off')

I_source.source_mode('CURR')
I_source.range(100e-3)
I_source.current(current*1.e-3)
I_source.output('on')


FSV.res_BW(FSV_rbw)
FSV.center_freq(FSV_center*1.e9)
FSV.averages(FSV_avg)
FSV.n_points(FSV_npt)
FSV.span(FSV_span)
FSV.ref_level(FSV_ref)
FSV.input_att(FSV_att)
FSV.write('SWE:TYPE AUTO')
FSV.sweep_time(FSV_time_sweep)

FSV.detector('rms')

print('\ntime estimation for all measurement : {} '
      .format(datetime.timedelta(seconds=2*2*FSV_time_sweep*len(src_frequency)*trace_average)))


MW_source.frequency(src_frequency[0]*1.e9)
MW_source.status(src_status.upper())
MW_source.power(src_power)
MW_source.set_gui_update('on')

parameter_snap={}
# # Snapshot parameters
# parameter_snap['twpa_pump'] = {'freq':pump_freq,'pump_power':pump_power,'pump_status' : pump_status,
#                             'current' : twpa_current}

# parameter_snap['MW_source'] = {'power':src_power}

parameter_snap['FSV']={'center_freq':FSV.center_freq(),'average':FSV_avg,'res_bandwidth':FSV_rbw,
                  'Npoints':int(FSV.n_points()),'ref':FSV_ref,'att':FSV_att,'span':FSV_span,'sweeptime':FSV_time_sweep}
parameter_snap['flux']={'current': current}

# Load experiment
exp_name = 'FSV_CW_vs_Agilent_freq'

sample_name = 'B5_G3_GALV_SQ200_1ST'

exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample_name)

measure(avg=trace_average,src_freq_vec=src_frequency,sa_freq=FSV_center,exp=exp, station=station)

# Shutdown devices
# VNA.rf_off()
# VNA.cont_meas_off()
MW_source.status('OFF')
I_source.output('off')



time estimation for all measurement : 2:46:40 
Starting experimental run with id: 3. 


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

2022-10-27 11:04:34,731 ¦ qcodes.dataset.measurements ¦ WARNING ¦ measurements ¦ __exit__ ¦ 659 ¦ An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-0184185fdbc4;
Traceback:
Traceback (most recent call last):
  File "C:\Users\nicolas.ROCH\AppData\Local\Temp\ipykernel_24712\3887396045.py", line 30, in measure
    get_s = get_s + np.average(1.e-3*10**(FSV.spectrum.get()/10))
  File "C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\parameter.py", line 648, in get_wrapper
    raw_value = get_function(*args, **kwargs)
  File "C:\QCodes_Driver_And_Scripts\Drivers\RS_FSV.py", line 35, in get_raw
    data = self._instrument.get_trace()
  File "C:\QCodes_Driver_And_Scripts\Drivers\RS_FSV.py", line 332, in get_trace
    sleep(1) # we wait until the register is 1
KeyboardInterrupt



KeyboardInterrupt: 

In [9]:
MW_source.status('OFF')
I_source.output('off')